# Árboles de decisión

<div style="text-align: right"><a>por </a><a href="https://www.linkedin.com/in/sheriff-data/" target="_blank">Manuel López Sheriff</a></div>

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

## Intro

Un árbol de decisión intenta predecir la variable objetivo utilizando una lógica como la siguiente:

<img width=600 src="https://miro.medium.com/v2/resize:fit:1084/1*U6twhNSe1I37feFfBaeXLw.png">

Árboles de decisión:
 * se utilizan tanto para la clasificación (ejemplo anterior Fit/Unfit) como para la regresión
 * implican estratificar (segmentar) el espacio de predictores...
 * de forma iterativa
 * reciben este nombre porque las reglas de división pueden resumirse en un árbol

Árboles de decisión:
 * son simples
 * son útiles para la interpretación
 * no son predictores muy potentes, pero...
 * dan lugar a modelos más complejos, como los algoritmos Random Forest o Gradient Boosted Trees

## El problema

Hoy utilizaremos un conjunto de datos de **vino blanco**.

Los expertos han calificado varios vinos, cuyas propiedades físicas también se indican

In [ ]:
df = pd.read_csv("../datasets/wine_quality.csv")

In [ ]:
df.shape

In [ ]:
df.sample(5)

### Exploración de datos

In [ ]:
# renombramos las columnas por comodidad
df.columns = [col.replace(" ", "_") for col in df.columns]

In [ ]:
df.sample(5)

Queremos:
 * construir un modelo de aprendizaje **supervisado**
 * de **regresión** (predecir variable cuantitativa)
 * que intenta predecir la `quality` del vino a partir de sus propiedades físicas (de modo que ya no necesitemos el asesoramiento de expertos)

In [ ]:
sns.countplot(x=df.quality, hue=df.quality, palette="Blues")

In [ ]:
df.quality.value_counts().sort_index()

In [ ]:
df.head()

In [ ]:
df.corr()["quality"].sort_values()

In [ ]:
sns.scatterplot(x=df.alcohol, y=df.quality)

In [ ]:
sns.boxplot(x=df.quality, y=df.alcohol, palette="Blues")

In [ ]:
df.groupby("quality").alcohol.median().sort_index()

In [ ]:
sns.scatterplot(x=df.fixed_acidity, y=df.volatile_acidity, alpha=0.1)

In [ ]:
sns.histplot(x=df.alcohol)

### Train / test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop("quality", axis=1)

In [ ]:
y = df.quality

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=98)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

### Paréntesis: construyamos una regresión lineal

Para evaluar correctamente el rendimiento del modelo, usaremos train/test split y la métrica RMSE

$$RMSE=\sqrt{\frac{1}{N} \sum(y - \hat{y})^2}$$

In [ ]:
from sklearn.metrics import mean_squared_error

1. Instancio una regresión lineal.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lin = LinearRegression()

2. La entreno con los 3918 vinos de train

In [ ]:
lin.fit(
    X=X_train, 
    y=y_train
)

In [ ]:
lin.intercept_

In [ ]:
pd.Series(lin.coef_, index=X.columns).sort_values()

3. Cojo un vino nuevo. Cómo se predice su `quality`?

In [ ]:
X_test[:5]

In [ ]:
lin.predict(X_test[:5]).round(2)

In [ ]:
y_test[:5]

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
# train error (MSE)
mean_squared_error(y_train, lin.predict(X_train), squared=False).round(3)

In [ ]:
# test error (MSE)
mean_squared_error(y_test, lin.predict(X_test), squared=False).round(3)

Suele funcionar mejor el modelo sobre el train

## Decision trees

Para evaluar correctamente el rendimiento del modelo, usaremos train/test split y la métrica RMSE

$$RMSE=\sqrt{\frac{1}{N} \sum(y - \hat{y})^2}$$

Vamos a:
 * probar varios modelos y...
 * nos quedaremos con el que tenga el **menor** RMSE en el **conjunto de test** (también llamado test error)
 * también mostraremos el error en el train

### Baseline model

El modelo baseline da la misma predicción para todos los vinos: la `quality` media

Es bueno evaluar este modelo para tenerlo de referencia

In [ ]:
baseline = y_train.mean()

In [ ]:
baseline

El RMSE puede calcularse manualmente

Train error

In [ ]:
(((y_train - baseline) ** 2).mean() ** 0.5).round(3)

Test error

In [ ]:
(((y_test - baseline) ** 2).mean() ** 0.5).round(3)

Como era de esperar, el error en el test es mayor que el del train

### Regresión lineal

Recordemos los resultados anteriores de la regresión lineal:

In [ ]:
# train error (MSE)
mean_squared_error(y_train, lin.predict(X_train), squared=False).round(3)

In [ ]:
# test error (MSE)
mean_squared_error(y_test, lin.predict(X_test), squared=False).round(3)

### Simple tree (depth=1)

Primero entrenemos el Decision Tree, luego interpretémoslo

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
model = DecisionTreeRegressor(max_depth=1, random_state=6666)

In [ ]:
model.fit(X_train, y_train)

Veamos cómo predice este modelo los 5 primeros vinos

In [ ]:
# real values
y_test[:5]

In [ ]:
# predicted values
model.predict(X_test[:5].values).round(2)

Train error

In [ ]:
mean_squared_error(
    y_true=y_train,
    y_pred=model.predict(X_train),
    squared=False
).round(3)

Test error

In [ ]:
mean_squared_error(
    y_true=y_test,
    y_pred=model.predict(X_test),
    squared=False
).round(3)

In [ ]:
from sklearn.tree import plot_tree

In [ ]:
df.head()

In [ ]:
y_train.mean()

In [ ]:
fig = plt.figure(figsize=(10, 6))
plot_tree(model, feature_names=df.columns[:-1], filled=True);

In [ ]:
mean_squared_error(y_train, model.predict(X_train)).round(3)

Es la media ponderada de los MSE de las hojas

Algunas preguntas importantes para una comprensión profunda:

 1. durante el entrenamiento, ¿por qué el árbol de decisión eligió la variable `alcohol` y el corte $10.85$?

Veamos qué hace el árbol en detalle para tomar esta decisión:

In [ ]:
df.head()

Imagina que elegimos `residual_sugar` y el valor 5

In [ ]:
group1 = X_train[X_train.residual_sugar <= 5].copy()
group2 = X_train[X_train.residual_sugar > 5].copy()

In [ ]:
group1.shape

In [ ]:
group2.shape

In [ ]:
group1_mean = y_train[group1.index].mean()

In [ ]:
group1_mean

In [ ]:
group2_mean = y_train[group2.index].mean()

In [ ]:
group2_mean

In [ ]:
mse = ((
    ((y_train[group1.index] - group1_mean) ** 2).sum() +
    ((y_train[group2.index] - group2_mean) ** 2).sum()
) / X_train.shape[0])

In [ ]:
mse

Mejora pequeña sobre el modelo de referencia, y mucho peor que el alcohol 10.85, que es el par (característica-umbral) óptimo

In [ ]:
fig = plt.figure(figsize=(10, 6))
plot_tree(model, feature_names=df.columns[:-1], filled=True);

In [ ]:
fig = plt.figure(figsize=(10, 6))
plot_tree(model, feature_names=df.columns[:-1], filled=True);

2. qué significa `squared_error`?  
   el error cuadrático medio en esa rama: el que se obtendría si a cada vino de esa rama se le diera la media de la rama

3. qué significa `value`?  
   la calidad media de los vinos de esa rama. El valor que se predecirá para cada nuevo vino que termine en esa hoja del árbol.

4. durante el testing (predicción de una nueva instancia), ¿cómo funciona el árbol?  
   Recorre un único camino. Cuando este camino termina, el valor en esa hoja es la predicción

### Bigger tree (depth=3)

In [ ]:
model = DecisionTreeRegressor(max_depth=3, random_state=666)

In [ ]:
%%time
model.fit(X_train, y_train)

In [ ]:
# real values
y_test[:15]

In [ ]:
# predicted values
model.predict(X_test[:15].values).round(2)

Train error

In [ ]:
mean_squared_error(
    y_true=y_train,
    y_pred=model.predict(X_train)
).round(3)

Test error

In [ ]:
mean_squared_error(
    y_true=y_test,
    y_pred=model.predict(X_test)
).round(3)

In [ ]:
fig = plt.figure(figsize=(25, 20))
plot_tree(model, feature_names=df.columns[:-1], filled=True);

In [ ]:
fig.savefig("depth3.svg")

### Huge tree (depth=20)

In [ ]:
model = DecisionTreeRegressor(max_depth=20, random_state=666)

In [ ]:
%%time
model.fit(X_train, y_train)

Train error

In [ ]:
mean_squared_error(
    y_true=y_train,
    y_pred=model.predict(X_train)
)

Test error

In [ ]:
mean_squared_error(
    y_true=y_test,
    y_pred=model.predict(X_test)
)

### Overfitting

Veamos cómo cambia el error de train y de test al ir variando `max_depth`

In [ ]:
results = []

for depth in range(1, 21):
    model = DecisionTreeRegressor(max_depth=depth, random_state=666)
    model.fit(X_train, y_train)
    
    result = {
        "model": model,
        "depth": depth,
        "train_error": mean_squared_error(y_train, model.predict(X_train)),
        "test_error": mean_squared_error(y_test, model.predict(X_test))
    }
    
    results.append(result)

In [ ]:
results_df = pd.DataFrame(results).round(3)

In [ ]:
results_df

In [ ]:
fig = plt.figure(figsize=(6, 4))
plt.plot(results_df.depth, results_df.train_error, label="train error")
plt.plot(results_df.depth, results_df.test_error, label="test error")
plt.legend()

Podemos ver cómo, cuando `max_depth` aumenta por encima de ~7:
 * el error de train sigue disminuyendo (más precisión en las muestras de entrenamiento)
 * el error de test aumenta (el modelo memoriza el conjunto de muestras de entrenamiento y no generaliza muy bien)

Es el famoso **overfitting**

Recuerda: el **test error** es el que hay que tener en cuenta para evaluar la performance de un modelo

In [ ]:
fig = plt.figure(figsize=(25,20))
plot_tree(results_df.loc[4].model, feature_names=df.columns[:-1], filled=True);

In [ ]:
fig.savefig("depth5.svg")

### Otros hiperparámetros

Además de `max_depth`, existen otros **hiperparámetros** que nos permiten construir diferentes arquitecturas de árboles de la familia DecisionTreeRegressor:

 * `min_samples_split`: el número mínimo de muestras necesarias para dividir un nodo

 * `max_features`: el número de features a considerar cuando se busca el mejor corte

In [ ]:
model = DecisionTreeRegressor(max_depth=7, min_samples_split=30, random_state=666)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
print(f"train error: {mean_squared_error(y_train, model.predict(X_train))}")
print(f"test error: {mean_squared_error(y_test, model.predict(X_test))}")

In [ ]:
fig = plt.figure(figsize=(25,20))
plot_tree(model, feature_names=df.columns[:-1], filled=True);

In [ ]:
fig.savefig("depth7-maxsplit30.svg")

### Grid search

Busquemos la *mejor* combinación de hiperparámetros, es decir, la que produzca el menor error de test, entre un grid prescrito de valores para cada hiperparámetro

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV

In [ ]:
gs = GridSearchCV(
    estimator=DecisionTreeRegressor(),
    param_grid={
        "max_depth": [6, 7, 8],
        "min_samples_split": [20, 30, 50, 100, 200],
    },
    cv=5,
    verbose=1,
    scoring="neg_mean_squared_error",
    return_train_score=True
)

Probará 3 * 5 = 9 opciones

In [ ]:
%%time
gs.fit(X_train, y_train)

Ordenemos todos los árboles según su rendimiento:

In [ ]:
grid_search_results = pd.DataFrame(gs.cv_results_)

grid_search_results = grid_search_results[['param_max_depth', 'param_min_samples_split',
       'mean_test_score', 'mean_train_score']]

In [ ]:
grid_search_results.sort_values("mean_test_score", ascending=False)

Podemos acceder al mejor estimador del Grid Search:

In [ ]:
best_tree = gs.best_estimator_

In [ ]:
best_tree

In [ ]:
mean_squared_error(best_tree.predict(X_train), y_train)

In [ ]:
mean_squared_error(best_tree.predict(X_test), y_test)

## Feature importance

¿Qué importancia tiene cada feature para predecir la `quality`?

DecisionTreeRegressor tiene un atributo `feature_importances_`

In [ ]:
best_tree

In [ ]:
feature_imp = pd.Series(best_tree.feature_importances_, index=df.columns[:-1]).sort_values(ascending=False)

In [ ]:
feature_imp.round(3)

In [ ]:
vinito = X_test[:1]

In [ ]:
vinito

In [ ]:
best_tree.predict(vinito)

In [ ]:
vinito_azucarado = vinito.copy()

In [ ]:
vinito_azucarado.iloc[0, -1] = 11

In [ ]:
best_tree.predict(vinito_azucarado)

In [ ]:
sns.barplot(x=feature_imp.values, y=feature_imp.index)

In [ ]:
fig = plt.figure(figsize=(20, 20))
plot_tree(best_tree, feature_names=df.columns[:-1], filled=True);

In [ ]:
fig.savefig("decision_tree.svg", facecolor="white")

## Resumen

 * Los árboles de decisión son útiles para la regresión (`DecisionTreeRegressor`) y la clasificación (`DecisionTreeClassifier`)
 * Su comportamiento es bastante intuitivo
 * Su comportamiento es interpretable y explicable

 * Los árboles de decisión tienen overfitting cuando `max_depth` se hace muy grande (obvio, muchas hojas con muy pocas muestras _memorizadas_ cada una)
 * Prevenir el overfitting (siempre, no sólo en los métodos basados en árboles) observando el error de train y el error de test

 * Un árbol de decisión no suele ser un algoritmo de ML muy potente
 * Los árboles de decisión son los componentes básicos de algoritmos más avanzados y potentes.